In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import pickle
from IPython.display import Image as IPyImage
from IPython.display import display
import tensorflow as tf
import re
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import pydot

2023-03-02 20:44:14.687781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 20:44:15.757946: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cudnn-8.2.4.15-11.4-eluwegpwn6adr7hlku5p5wru5xzefpop/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cuda-11.4.4-ctldo35wmmwws3jbgwkgjjcjawddu3qz/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/neovim-0.7.0-terkir3wk5rst6ktv4uxyaqjditacv5p/lib
2023-03-02 20:44:15.758030: W tensorflow/compiler/xla/stream_execut

### Constants

In [2]:
IMAGES_DIRECTORY = "/work/users/skoka/Data/flickr30k_images/flickr30k_images/"
LABELS_FILE = "/work/users/skoka/Data/flickr30k_images/results.csv"
PICKLE_DIRECTORY = "/work/users/skoka/Data/flicker30k_pickles/"
GET_FROM_PICKLE = True

In [3]:
# reading in the labels
labels = pd.read_csv(LABELS_FILE, delimiter='|')
# convert labels[ ' comment' ] to strings
labels[' comment'] = labels[' comment'].astype(str)

# list of .jpg files from the directory
image_files = [f for f in os.listdir(IMAGES_DIRECTORY) if f.endswith('.jpg')]

In [4]:
# Average number of words in a sentence
avg_words = 0
for i in range(len(labels)):
    avg_words += len(labels[' comment'][i].split())
avg_words = avg_words / len(labels)
print("Average number of words in a sentence: ", avg_words)

Average number of words in a sentence:  13.389340213321587


In [5]:
# Converts a jpg file to a numpy array
def _read_jpg(filename):
    im = Image.open(filename)
    # resize to 224x224
    im = im.resize((224, 224))
    return np.array(im)

def process_files(files):
    images = {
        # "filename_without_extension" : numpy_array_of_image
    }
    num_read = 0
    for f in files:
        image_name = f.split('.')[0]
        image = _read_jpg(IMAGES_DIRECTORY + f)
        images[image_name] = {
            "image": _read_jpg(IMAGES_DIRECTORY + f),
            "captions": labels[labels.image_name == (image_name + ".jpg")][" comment"].tolist()
            }
        num_read += 1
        if num_read % 1000 == 0:
            print("Read {} files".format(num_read))
    return images
if not GET_FROM_PICKLE:
    images = process_files(image_files)
    with open(PICKLE_DIRECTORY + "flicker_images.pkl", "wb") as f:
        pickle.dump(images, f)
else:
    with open(PICKLE_DIRECTORY + "flicker_images.pkl", "rb") as f:
        images = pickle.load(f)

### Data Formatting
{
    "image_name" : {
        "image" : "image", // Numpy array of the image
        "captions" :  [
            "caption 1",
            "caption 2",
            ...
        ]
    }
}

In [39]:
from keras.preprocessing.text import Tokenizer
captions = labels[" comment"].to_numpy()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions)
tokenizer.fit_on_texts(['staaaart', 'endddd'])
vocab_size = len(tokenizer.word_index) + 1
# tokenizer.word_index['staaaart'] = vocab_size -1
# tokenizer.word_index['endddd'] = vocab_size
word_index = tokenizer.word_index

In [40]:
len(word_index)

18315

In [15]:
import time
MAX_CAPTION_LENGTH = 20
def clean_caption(caption):
    caption = caption.lower()
    # remove non alphanumeric characters
    caption = re.sub(r'[^a-zA-Z0-9\s]', '', caption)
    # add start and end tokens
    caption = 'staaaart ' + caption + ' endddd'
    return caption
total_time = 0
def data_generator(batch_size=32):
    global total_time
    start = time.time()
    X1, X2, y = [], [], []
    n = 0
    while True:
        for image_name, image_data in images.items():
            image = image_data["image"]
            captions = image_data["captions"]
            for caption in captions:
                caption = clean_caption(caption)
                caption = tokenizer.texts_to_sequences([caption])[0]
                for i in range(1, len(caption)):
                    in_seq, out_seq = caption[:i], caption[i]
                    in_seq = pad_sequences([in_seq], maxlen=MAX_CAPTION_LENGTH)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    X1.append(image)
                    X2.append(in_seq)
                    y.append(out_seq)
                    n += 1
                    if n == batch_size:
                        X1 = np.array(X1)
                        X1 = X1 / 255
                        X2 = np.array(X2)
                        y = np.array(y)
                        total_time += time.time() - start
                        yield [X1, X2], y
                        start = time.time()
                        X1, X2, y = [], [], []
                        n = 0

In [16]:
def load_embeddings(filename, embed_size):
    # the embed size should match the file you load glove from
    embeddings_index = {}
    f = open(filename)
    # save key/array pairs of the embeddings
    #  the key of the dictionary is the word, the array is the embedding
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

    # now fill in the matrix, using the ordering from the
    #  keras word tokenizer from before
    found_words = 0
    embedding_matrix = np.zeros((len(word_index) + 1, embed_size))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be ALL-ZEROS
            embedding_matrix[i] = embedding_vector
            found_words = found_words+1

    print("Embedding Shape:",embedding_matrix.shape, "\n",
        "Total words found:",found_words, "\n",
        "Percentage:",100*found_words/embedding_matrix.shape[0])
    return embedding_matrix

# embedding_matrix = load_embeddings("/users/skoka/Documents/ML-Lab2-Multi-Modal/numberbatch-en-19.08.txt", 300)

In [17]:
# Build a VGG model that takes input of size 250x250x3
vgg = tf.keras.applications.VGG19(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# remove the last layer of the VGG model
vgg = tf.keras.Model(inputs=vgg.inputs, outputs=vgg.layers[-2].output)
vgg.trainable = False


image_dense = tf.keras.layers.Flatten()(vgg.output)
image_dense = tf.keras.layers.Dense(256, activation='relu')(image_dense)
image_reshaped = tf.keras.layers.Reshape((1, 256))(image_dense)

caption_input = tf.keras.layers.Input(shape=(MAX_CAPTION_LENGTH,))
caption_embedding = tf.keras.layers.Embedding(vocab_size, 256, mask_zero=False)(caption_input)

# Merge the two models
decoder_add = tf.keras.layers.add([image_reshaped, caption_embedding], axis=1)

# bi directonal LSTM
decoder_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=False))(decoder_add)
decoder2 = tf.keras.layers.Dense(256, activation='relu')(decoder_lstm)
outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(decoder2)

model = tf.keras.Model(inputs=[vgg.input, caption_input], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# plot model:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                           

In [18]:
# plot model:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [56]:
history = model.fit_generator(data_generator(batch_size=64),
                                epochs=1,
                                steps_per_epoch=1000,
                                callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.00000001, verbose=1)],
                                verbose=1)

   1/1000 [..............................] - ETA: 1:34 - loss: 3.8221

/tmp/ipykernel_2874860/3685102400.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(data_generator(batch_size=64),


1000/1000 [==============================] - 39s 39ms/step - loss: 3.9230 - lr: 0.0010


In [ ]:
# save model
model.save('model.h5')

In [54]:
def predict_caption(model, image):
    in_text = 'staaaart'
    for i in range(MAX_CAPTION_LENGTH):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=MAX_CAPTION_LENGTH)
        print(sequence)
        yhat = model.predict([image, sequence], verbose=0)
        # print top 5 predicted words:
        yhat = np.argmax(yhat)
        word = tokenizer.index_word[yhat]
        in_text += ' ' + word
        if word == 'endddd':
            break
    return in_text

In [59]:
# Predict on a random image
import random
image_name = random.choice(list(images.keys()))
image = images[image_name]["image"]
caption = images[image_name]["captions"][0]
print("Actual Caption:", caption)
predicted_caption = predict_caption(model, image.reshape(1, 224, 224, 3))
print("Predicted Caption:", predicted_caption)

Actual Caption:  A man in a white t-shirt does a trick with a bronze colored yo-yo .
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0 18314]]
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 18314     1]]
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0 18314     1     6]]
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0 18314     1     6     2]]
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0 18314     1     6     2     1]]
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0 18314     1     6     2     1    24]]
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0 18314     1     6     2     1    24    18]]
[[    0     0     0     0     0  